In [1]:
import pandas as pd
import numpy as np
import glob
import os

In [3]:
df = pd.read_csv('train/train.csv')
actions = pd.read_csv('messages/actions.csv')
messages = pd.read_csv('messages/messages.csv')
addresses = pd.read_csv('misc/addresses.csv')
mc = pd.read_csv('misc/master_categories.csv')
up = pd.read_csv('misc/user_profiles.csv')

In [158]:
ss = pd.read_csv('sample_submission_from_kaggle_change_sep.csv').drop(['Predicted'], 1)
ss

,Id
0,19843
1,471287
2,342522
3,457799
4,233778
...,...
214604,16841
214605,69348
214606,594974
214607,320024


In [231]:
path = 'shipments/'
all_files = glob.glob(os.path.join(path, "*.csv"))
use_cols = ["user_id", "ship_address_id", "shipment_id", "order_id", "order_completed_at", "total_cost", "promo_total", "total_weight", "os", "s.order_state", 'rate']

df_from_each_file = (pd.read_csv(f, usecols=use_cols, parse_dates=['order_completed_at']) for f in all_files)
shipments = pd.concat(df_from_each_file, ignore_index=True)  # create one shipments table from all .csv in path
shipments = shipments[shipments['s.order_state'] == 'complete'].drop(['s.order_state'], 1)  # take only `complete` orders
shipments.drop(index=shipments.query('order_completed_at < 2020.01').index, inplace=True)  # drop shipments before 2020

In [5]:
df

,phone_id,order_completed_at,target
0,19843,2020-01,1.0
1,19843,2020-02,0.0
2,19843,2020-03,0.0
3,19843,2020-04,0.0
4,19843,2020-05,0.0
...,...,...,...
1159571,23853,2020-03,0.0
1159572,23853,2020-04,0.0
1159573,23853,2020-05,0.0
1159574,23853,2020-06,1.0


In [196]:
df_pivot = df.pivot(index='phone_id', columns='order_completed_at', values='target')
df_pivot.index.rename('Id', inplace=True)
df_pivot.head()

order_completed_at,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07
Id,,,,,,,
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,0.0,1.0,1.0,1.0,1.0,0.0,0.0
6,1.0,1.0,1.0,0.0,1.0,0.0,0.0
8,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [225]:
train_id = df_pivot.index
test_id = ss.Id
all_id = set(train_id) | set(test_id)

In [224]:
addresses_gb = addresses.groupby('phone_id').agg({'id': list}).query('index in @all_id')
addresses_gb  # all addresses for each phone_id

,id
phone_id,
2,"[8531, 2982657, 3271801, 3648206, 3921697]"
3,"[17410, 6697340, 4020938, 4021954]"
4,"[23364, 6451527, 8418512, 7144833, 7608581, 48..."
6,"[45029, 2820225, 8287431, 2895360, 2914052, 32..."
8,"[49164, 3760106]"
...,...
719359,[11723842]
719360,[11723857]
719363,[11723970]


In [228]:
all_ship_address_id = set(el for l in addresses_gb.id.values for el in l)  # set of ship_address_id

In [232]:
shipments.query('ship_address_id in @all_ship_address_id', inplace=True)
shipments

,user_id,ship_address_id,shipment_id,order_id,order_completed_at,total_cost,rate,promo_total,total_weight,os
0,11019,171906,322307,2156687,2020-02-18 14:07:00,168.0,0,0.0,30170,windows
1,62278,278832,387023,3021953,2020-01-03 13:09:26,98.0,0,-150.0,11305,windows
2,905126,468407,388943,3018198,2020-02-12 12:39:28,98.0,5,0.0,13589,ios
3,21412,61962,421048,3030227,2020-01-25 11:58:56,158.0,0,0.0,9726,mac
4,42110,378297,442659,2923996,2020-01-07 14:30:44,163.0,0,0.0,30323,windows
...,...,...,...,...,...,...,...,...,...,...
2260677,2845749,12168752,9916493,15511307,2020-08-31 23:45:52,158.0,0,0.0,12266,android
2260679,1813646,11858902,9916509,15167417,2020-08-31 23:54:18,116.0,0,0.0,52320,android
2260680,874653,11893429,9916515,15206448,2020-08-31 23:56:44,158.0,0,0.0,1915,ios
2260681,274733,12102714,9916517,15438866,2020-08-31 23:56:47,199.0,0,-250.0,3990,windows


Grouped orders by each month. There are 8 dataframes for each month from Jan to Aug. List of them stored in `shipments_months`

In [233]:
suf = ['_jan', '_feb', '_mar', '_apr', '_may', '_jun', '_jul', '_aug']

shipments_months = [g for _, g in shipments.groupby(pd.Grouper(key='order_completed_at', freq='M'))]
shipments_months = [g.groupby('ship_address_id')[['total_cost', 'promo_total', 'total_weight', 'rate']].mean().add_suffix(s) for g,s in zip(shipments_months, suf)]

In [234]:
def f(x, month=0):
    '''
    param x: phone_id from `addresses_gb` table
    param month: int fom 0 to 7 (Jan to Aug)
    return: mean values of 'total_cost', 'promo_total', 'total_weight', 'rate' for all ship_address_id for given phone_id in given month
    '''
    return shipments_months[month].query('index in @list(@x.values)[0]').mean()

def g(month):
    '''
    Used previous funtion f and return mean values of 'total_cost', 'promo_total', 'total_weight', 'rate' for each row in `addresses_gb` table
    param month: int fom 0 to 7 (Jan to Aug)
    '''
    return addresses_gb.apply(lambda x: f(x, month), axis=1)

In [235]:
from joblib import Parallel, delayed

In [236]:
%%time
shipments_all = pd.concat(Parallel(n_jobs=-1)(delayed(g)(month) for month in range(8)), 1)  # for faster parallel queries

Wall time: 2h 39min 49s


In [237]:
shipments_all

,total_cost_jan,promo_total_jan,total_weight_jan,rate_jan,total_cost_feb,promo_total_feb,total_weight_feb,rate_feb,total_cost_mar,promo_total_mar,...,total_weight_jun,rate_jun,total_cost_jul,promo_total_jul,total_weight_jul,rate_jul,total_cost_aug,promo_total_aug,total_weight_aug,rate_aug
phone_id,,,,,,,,,,,,,,,,,,,,,
2,98.0,0.0,22024.0,0.0,183.0,0.0,33973.0,0.0,233.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,...,NaN,NaN,0.0,-200.0,28032.0,5.0,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,158.0,0.0,...,28358.5,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,158.0,0.0,6236.0,5.0,198.0,0.0,32444.0,0.0,158.0,0.0,...,46266.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,158.0,0.0,21500.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
719359,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,-503.649994,29775.0,5.0
719360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,-156.449997,7400.0,0.0
719363,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,7200.0,5.0


In [238]:
shipments_all.to_csv('shipments_all.csv')

In [5]:
shipments_all = pd.read_csv('shipments_all.csv')

In [6]:
X = df_pivot.join(shipments_all, how='inner').fillna(0)
X

,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,phone_id,total_cost_jan,promo_total_jan,...,total_weight_jun,rate_jun,total_cost_jul,promo_total_jul,total_weight_jul,rate_jul,total_cost_aug,promo_total_aug,total_weight_aug,rate_aug
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,4,0.0,0.0,...,28358.5,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.0,1.0,0.0,1.0,0.0,1.0,0.0,6,158.0,0.0,...,46266.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.0,1.0,1.0,1.0,1.0,0.0,0.0,8,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
6,1.0,1.0,1.0,0.0,1.0,0.0,0.0,10,0.0,0.0,...,4000.0,0.0,0.0,0.0,0.0,0.0,158.0,0.000000,3150.0,0.0
8,1.0,0.0,1.0,0.0,0.0,0.0,0.0,12,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310248,0.0,1.0,0.0,0.0,1.0,0.0,1.0,719321,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,16321.0,5.0
310250,0.0,0.0,0.0,0.0,0.0,1.0,0.0,719347,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,350.0,0.000000,100000.0,0.0
310253,0.0,0.0,0.0,0.0,0.0,1.0,0.0,719363,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,7200.0,5.0
310254,0.0,0.0,0.0,0.0,0.0,1.0,1.0,719373,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-193.270004,18470.0,0.0


In [40]:
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
sc = StandardScaler()

In [246]:
# X_train, X_val = train_test_split(X.iloc[:, :-4], random_state=42, test_size=0.2)

In [247]:
# X_train, y_train = sc.fit_transform(X_train.iloc[:, 7:].values).reshape(X_train.shape[0], 7, -1), X_train.iloc[:, :7].values
# X_val, y_val = sc.transform(X_val.iloc[:, 7:].values).reshape(X_val.shape[0], 7, -1), X_val.iloc[:, :7].values

In [ ]:
# X_train, y_train = sc.fit_transform(X.iloc[:, 7:].values).reshape(X.shape[0], 7, -1), X.iloc[:, :7].values

Take first seven months for training (from Jan to Jul), and last seven for test (from Feb to Aug). For each month we have 4 numerical values.

In [21]:
X_, y_train = sc.fit_transform(X.iloc[:, 8:].values), X.iloc[:, :7].values
X_train, X_test = X_[:, :-4].reshape(X_.shape[0], 7, -1), X_[:, 4:].reshape(X_.shape[0], 7, -1)

In [26]:
def recall(y_true, y_pred):
    y_true = K.ones_like(y_true) 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    all_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    
    recall = true_positives / (all_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    y_true = K.ones_like(y_true) 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_score_(y_true, y_pred):
    pr = precision(y_true[:, -1], y_pred[:, -1])
    rec = recall(y_true[:, -1], y_pred[:, -1])
    return 2*((pr*rec)/(pr+rec+K.epsilon()))

In [197]:
opt = Adam(learning_rate=1e-2)

model = Sequential()
model.add(LSTM(128, input_shape=X_train.shape[1:], return_sequences=True))
model.add(LSTM(128))
model.add(Dense(y_train.shape[1], activation='sigmoid'))
model.compile(loss='bce', optimizer=opt, metrics=['accuracy', f1_score_])
model.fit(X_train, y_train, epochs=300, batch_size=1024, verbose=1)

Epoch 1/300
195/195 [==============================] - 5s 28ms/step - loss: 0.5681 - accuracy: 0.2258 - f1_score_: 0.3388
Epoch 2/300
195/195 [==============================] - 5s 28ms/step - loss: 0.5646 - accuracy: 0.2350 - f1_score_: 0.3317
Epoch 3/300
195/195 [==============================] - 5s 28ms/step - loss: 0.5645 - accuracy: 0.2353 - f1_score_: 0.3539
Epoch 4/300
195/195 [==============================] - 5s 28ms/step - loss: 0.5642 - accuracy: 0.2362 - f1_score_: 0.3446
Epoch 5/300
195/195 [==============================] - 6s 29ms/step - loss: 0.5642 - accuracy: 0.2357 - f1_score_: 0.3729
Epoch 6/300
195/195 [==============================] - 5s 28ms/step - loss: 0.5639 - accuracy: 0.2368 - f1_score_: 0.3789
Epoch 7/300
195/195 [==============================] - 5s 28ms/step - loss: 0.5638 - accuracy: 0.2355 - f1_score_: 0.3873
Epoch 8/300
195/195 [==============================] - 5s 28ms/step - loss: 0.5637 - accuracy: 0.2368 - f1_score_: 0.3671
Epoch 9/300
195/195 [===

In [84]:
scores = model.evaluate(X_train, y_train, verbose=2)

6212/6212 - 15s - loss: 0.4361 - accuracy: 0.2757 - f1_score_: 0.4462


In [198]:
y_pred = model.predict(X_train)[:, -1]

In [199]:
(y_pred > 0.5).astype(int)[:20]

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1])

In [200]:
y_train[:20, -1].astype(int)

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1])

In [103]:
import scipy as sc

In [201]:
os = sc.optimize.minimize_scalar(lambda x: -f1_score(y_train[:, -1].astype(int), y_pred > x), bounds=[0,1], method='bounded')
os  # for finding best threshold

     fun: -0.6993582556003531
 message: 'Solution found.'
    nfev: 23
  status: 0
 success: True
       x: 0.3311005599932239

In [202]:
y_subm = model.predict(X_test)[:, -1] > os.x  # predicting and take only last month for submission
X['Predicted'] = y_subm
X.rename(columns={'phone_id': 'Id'}, inplace=True)

In [251]:
ss.merge(X[['Id', 'Predicted']], how='left').fillna(0).astype(int).to_csv('subm_fillna0_300epoch.csv', index=False)

In [248]:
merged = ss.merge(X[['Id', 'Predicted']], how='left')
merged

,Id,Predicted
0,19843,True
1,471287,NaN
2,342522,NaN
3,457799,False
4,233778,False
...,...,...
214604,16841,True
214605,69348,NaN
214606,594974,True
214607,320024,True


In [223]:
def get_last_target(phone_id):
    if phone_id in df_pivot.index:
        return df_pivot.loc[phone_id, '2020-07']
    return 0

In [249]:
merged.loc[merged.isna().any(1), 'Predicted'] = merged.loc[merged.isna().any(1)].apply(lambda x: get_last_target(x.Id), 1)
merged.astype(int).to_csv('subm_fillna_last_seen.csv', index=False)

In [250]:
merged

,Id,Predicted
0,19843,True
1,471287,1
2,342522,0
3,457799,False
4,233778,False
...,...,...
214604,16841,True
214605,69348,0
214606,594974,True
214607,320024,True


In [108]:
messages

,sent,action_id,user_id
0,1.588847e+09,1512.0,1775692
1,1.588847e+09,1512.0,1775687
2,1.588847e+09,1512.0,1775684
3,1.588847e+09,1512.0,1775678
4,1.588847e+09,1512.0,1775660
...,...,...,...
18275902,1.597229e+09,2515.0,2551475
18275903,1.597229e+09,2515.0,2465134
18275904,1.597228e+09,2515.0,2472800
18275905,1.597227e+09,2515.0,2551294


In [8]:
actions.head()

,id,subject,body,type
0,239,NaN,NaN,push
1,77,NaN,NaN,push
2,213,Бесплатно доставим вам первый заказ👍,"Оформляйте скорее, доставим в удобное время!",push
3,218,👍 Новые скидки METRO в Instamart,Успей заказать продукты по выгодной цене!,push
4,221,Дарим 500 рублей за приглашенного друга!,"Обнови приложение, получи промокод и делись им...",push


In [12]:
up

,user_id,gender,bdate
0,2226207,NaN,NaN
1,2226207,NaN,NaN
2,2226207,NaN,NaN
3,1684318,male,1987-10-11
4,1684318,male,1987-10-11
...,...,...,...
396815,1253727,NaN,NaN
396816,1253727,NaN,NaN
396817,1253727,NaN,NaN
396818,1253727,NaN,NaN


In [11]:
mc

,id,name,parent_category_id
0,1,Продукты питания,0
1,2,Замороженные продукты,1
2,3,Замороженные овощи и фрукты,2
3,4,Замороженные полуфабрикаты,2
4,5,Рыба замороженная,2
...,...,...,...
1011,1012,Перевязочные материалы,571
1012,1013,Расходные материалы,571
1013,1014,Диагностические тесты,571
1014,1015,Диагностические приборы,571
